In [17]:
import ccxt
import time
from collections import deque
import datetime
import pandas as pd
import filter
import login

# 내 집갑의 원화 잔고 조회
def get_balance_krw(balances):
    return print('내 지갑 잔고 = ', balances[0]['balance'])

# 내 지갑의 코인 잔고 조회
def get_balance_coin(balances):
    b = True
    while (b):
        if balances[1]['currency'] == 'BTC/USDT':
            balance = balances[1]['balance']
            avg_buy_price = balances[1]['avg_buy_price']
            c = (float(avg_buy_price), float(balance))
            b = False
        else:
            c = (int(0), int(0))
            b = False
    return c

def get_ticker_ma(volume, df, balances, df_2):

    ma10 = df_2['close'].rolling(window=10).mean()
    df_2.insert(len(df_2.columns), "MA10", ma10)

    ma25 = df_2['close'].rolling(window=25).mean()
    df_2.insert(len(df_2.columns), "MA25", ma25)

    ma50 = df_2['close'].rolling(window=50).mean()
    df_2.insert(len(df_2.columns), "MA50", ma50)
    
    now_price = df.iloc[-1]['close']                         # 코인의 현재가
    coin_check = get_balance_coin(balances) # 코인 보유 하고 있는지 체크
    avg_price = coin_check[0]                          # 매수 평균가
    balance = coin_check[1]                             # 코인 보유 개수
    
    # 매수 평균가가 int 이면 매수 조건 체크 float이면 매도 조건 체크
    if type(avg_price) == int:

        if df_2.iloc[-1]['MA10'] > df_2.iloc[-1]['MA25'] and now_price > df_2.iloc[-1]['MA50']:
            # 살 가격 정하기
            buy_price = volume
            buy_order(buy_price, balances)
        else:
            print('기다리는 중')
            pass
    else:
        # 현재 보유 코인 수익률 계산
        buy_profit = ((now_price - avg_price)/avg_price) * 100
        profit = round(buy_profit, 2)

        if df_2.iloc[-1]['MA10'] < df_2.iloc[-1]['MA25'] or avg_price > now_price:
            sell_order(balances)
        else:
            print(f"코인명: BTC/USDT, 수익률: {profit}%")
            print("내가산 가격 = {}, 시장가격 = {}".format(avg_price, now_price))

# 매수 주문
def buy_order(buy_price, balances):
    while True:
        # 8000원을 빼고 
        print("내가 산 금액 = {:,.0f}원".format(buy_price))
        balances[0]['balance'] = float(balances[0]['balance']) - buy_price
        balances[1]['currency'] = 'BTC/USDT'
        # 8000원을 종가로 나누어 코인갯수 구하기
        balances[1]['balance'] = float(buy_price / df.iloc[-1]['close'])
        #샀을때의 가격
        balances[1]['avg_buy_price'] = float(df.iloc[-1]['close'])
        print('매수 체결')
        return

# 매도 주문
def sell_order(balances):
    while True:
        # 내가 가지고 있는 코인 갯수를 종가와 곱해서 팔았을때의 가격 측정
        sell_price = df.iloc[-1]['close'] * (balances[1]['balance'])
        print("내가 판 금액 = {:,.0f}원".format(sell_price))
        balances[1]['currency'] = ''
        balances[1]['balance'] = ''
        balances[1]['avg_buy_price'] = ''
        # 팔았을 때의 가격을 기존의 지갑잔고에 더하기
        balances[0]['balance'] = balances[0]['balance'] + sell_price
        print('매도 체결')
        print('*** 내 지갑 잔고 = {:,.0f}원 ***'.format(balances[0]['balance']))
        return

# 로그인

binance = login.login()
i = 0
tickers = []
ma10 = []
ma25 = []
ma50 = []
balances = []
# 시작 날짜 입력
day = datetime.datetime(2021,1,1)
day_2 = day
# 주식차트 기준 시간 입력
interval = '1h'
interval_2 = '1d'
# day = 24, minute240 = 4
time_select = 1
time_select_2 = 24
# 초기 지갑 잔고 입력
volume = 10000000
# 초기 지갑설정
balances = ({'currency': 'dolor', 'balance': '{}'.format(volume), 'avg_buy_price': ''}, {'currency': '', 'balance': '', 'avg_buy_price': ''})
while True:
    #ticker = filter.filter_1()
    #ticker = filter.filter_2(ticker)
    #ticker = filter.filter_3(ticker)
    # interval에 종목시간 입력
    day = int(time.mktime(day.timetuple())*1000)
    df = binance.fetch_ohlcv(symbol = 'BTC/USDT', since = day, limit= 50, timeframe = interval)
    df_2 = binance.fetch_ohlcv(symbol = 'BTC/USDT', since = day, limit= 50, timeframe = interval_2)
    df = pd.DataFrame(df, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])
    df_2 = pd.DataFrame(df, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])
    day = datetime.datetime.fromtimestamp(int(day)/1000)
    # 시간에 24시간씩 계속 추가
    print('---------------------------------------------')
    print(day)
    day = day + datetime.timedelta(hours = time_select)
    if i == 24:
        i = 0
        day_2 = day_2 + datetime.timedelta(hours = time_select_2)
    volume = float(balances[0]['balance'])
    if day == datetime.datetime(2022, 1, 1):
        if balances[1]['currency'] == 'BTC/USDT':
            final_balances = float(balances[0]['balance']) + float(balances[1]['balance']) * float(df['close'][-1])
        else:
            final_balances = float(balances[0]['balance'])
        print('*** 내 지갑 잔고 = {:,.0f}원 ***'.format(final_balances))
        break
    get_ticker_ma(volume, df, balances, df_2)
    i += 1
    time.sleep(0.1)


---------------------------------------------
2021-01-01 00:00:00
내가 산 금액 = 10,000,000원
매수 체결
---------------------------------------------
2021-01-01 01:00:00
코인명: BTC/USDT, 수익률: 1.11%
내가산 가격 = 32543.45, 시장가격 = 32904.2
---------------------------------------------
2021-01-01 02:00:00
코인명: BTC/USDT, 수익률: 1.71%
내가산 가격 = 32543.45, 시장가격 = 33100.0
---------------------------------------------
2021-01-01 03:00:00
코인명: BTC/USDT, 수익률: 1.76%
내가산 가격 = 32543.45, 시장가격 = 33115.41
---------------------------------------------
2021-01-01 04:00:00
내가 판 금액 = 9,498,686원
매도 체결
*** 내 지갑 잔고 = 9,498,686원 ***
---------------------------------------------
2021-01-01 05:00:00
내가 산 금액 = 9,498,686원
매수 체결
---------------------------------------------
2021-01-01 06:00:00
코인명: BTC/USDT, 수익률: 1.41%
내가산 가격 = 31722.66, 시장가격 = 32171.14
---------------------------------------------
2021-01-01 07:00:00
코인명: BTC/USDT, 수익률: 1.5%
내가산 가격 = 31722.66, 시장가격 = 32199.91
---------------------------------------------
2021-01-01 08

In [3]:
df.iloc[-1]['close']

9533.32